In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from dechorate import constants
from dechorate.utils.file_utils import load_from_pickle

# Load RIRs

In [3]:
path_to_hdf = Path('..', 'dEchorate_rirs.hdf5')
dset = h5py.File(path_to_hdf, mode='r')

signals = list(dset.keys())
signal = signals[0]
print('Signals', signals)

rooms = list(dset[signals[0]].keys())
print('Rooms', rooms)

sources = list(dset[signals[0]][rooms[0]].keys())
print(sources)

Signals ['rir']
Rooms ['000000', '000001', '000010', '000100', '001000', '010000', '011000', '011100', '011110', '011111', '020002']
['1', '2', '3', '4', '5', '6', '7', '8', '9']


In [4]:
path_to_delay_note = Path('..','data', 'dEchorate_database.csv')
df = pd.read_csv(path_to_delay_note)

In [5]:
# build RIR matrix
D = len(rooms)
J = len(sources)
I = 30
Fs = constants['Fs']
L = int(2*Fs)

rirs = np.zeros([L, I, J, D])

for r, room in enumerate(rooms):

    for j, src in enumerate(sources):

        group = f'/{signal}/{room}/{src}'
        data = np.asarray(dset[group])
        
        rirs[:,:,j,r] = data[:2*Fs,:-1]
        
print(rirs.shape)

(96000, 30, 9, 11)


In [6]:
echo_notes = load_from_pickle('/home/chutlhu/Documents/Datasets/dEchorate/raw/processed/mics_srcs_echo_annotation.pkl')
print(echo_notes['toa_pck'].shape)

echo_df = pd.DataFrame()
c = 0

refl_order_calibr = ['direct', 'ceiling', 'floor', 'west', 'south', 'east', 'north']

for i in range(30):
    for j in range(4):
        for w in range(7):            
            toa = echo_notes['toa_pck'][w,i,j]
            wall = refl_order_calibr[w]
            
            echo_df.at[c, 'mic'] = i
            echo_df.at[c, 'arr'] = i//5
            echo_df.at[c, 'src'] = j
            echo_df.at[c, wall] = toa
            
        c += 1
            
print(echo_df)
echo_df.to_csv('../outputs/echo_pickpeaking.csv')

toa_pck = echo_notes['toa_pck']
toa_sym = echo_notes['toa_sym']
print(toa_pck.shape)
print(toa_sym.shape)
        

(7, 30, 4)
      mic  arr  src    direct   ceiling     floor      west     south  \
0     0.0  0.0  0.0  0.003911  0.008312  0.025258  0.025777  0.010918   
1     0.0  0.0  1.0  0.008970  0.011105  0.027283  0.013691  0.019400   
2     0.0  0.0  2.0  0.011240  0.015636  0.017760  0.020040  0.020001   
3     0.0  0.0  3.0  0.011090  0.015848  0.017359  0.024534  0.016562   
4     1.0  0.0  0.0  0.003803  0.008418  0.025148  0.025805  0.010845   
..    ...  ...  ...       ...       ...       ...       ...       ...   
115  28.0  5.0  3.0  0.004827  0.021994  0.011101  0.021636  0.014363   
116  29.0  5.0  0.0  0.003943  0.014292  0.019361  0.023516  0.011506   
117  29.0  5.0  1.0  0.008767  0.015283  0.021495  0.013282  0.021640   
118  29.0  5.0  2.0  0.005964  0.021167  0.012877  0.018330  0.018330   
119  29.0  5.0  3.0  0.005042  0.021752  0.011329  0.021704  0.014288   

         east     north  
0    0.008096  0.007425  
1    0.010810  0.011748  
2    0.012997  0.013497  
3    0.0

# RIRs to SOFAR's DRIRS

In [7]:
import sofar as sf
sf.list_conventions()

Available SOFA conventions:
SimpleFreeFieldHRTF (Version 2.0)
SingleRoomDRIR (Version 0.3)
MultiSpeakerBRIR (Version 0.3)
GeneralFIRE (Version 1.0)
FreeFieldDirectivityTF (Version 1.0)
SimpleHeadphoneIR (Version 1.0)
SimpleFreeFieldHRIR (Version 1.0)
GeneralFIR-E (Version 2.0)
SingleRoomSRIR (Version 1.0)
SimpleFreeFieldHRSOS (Version 1.0)
GeneralTF-E (Version 1.0)
GeneralFIR (Version 1.0)
SimpleFreeFieldSOS (Version 1.0)
SingleRoomMIMOSRIR (Version 1.0)
GeneralTF (Version 2.0)
FreeFieldHRTF (Version 1.0)
FreeFieldHRIR (Version 1.0)
GeneralFIR (Version 2.0)
GeneralTF (Version 1.0)



Documentation available at [sofaconventions.org](https://www.sofaconventions.org/mediawiki/index.php/SOFA_conventions)

- **SingleRoomSRIR**: spatial room impulse responses (SRIRs) measured with an arbitrary number of receivers (R>1, such as a microphone array) and an omnidirectional source (E=1) in a single room. SingleRoomSRIR is the standardized version of the previously proposed SingleRoomDRIR.
- **SingleRoomMIMOSRIR**: SRIRs measured with a compact listener containing an arbitrary number of receivers (R>1, such as a compact microphone array) and a compact source containing an arbitrary number of emitters (E>1, such as a multi-emitter loudspeaker) for multiple positions and/or orientations of the listener and/or source, in a single room.

We go for **SingleRoomMIMOSRIR**, one object for array

In [8]:
sofa = sf.Sofa('SingleRoomSRIR')
sofa

sofar.SOFA object: SingleRoomSRIR 1.0

In [9]:
# current params
room_idx = 1
print(room_idx)
arr_idx = 0
print(arr_idx)
mic_idxs = np.arange(5*arr_idx,5*(arr_idx+1))
print(mic_idxs)
src_idx = 0
print(src_idx)
print(rirs.shape)

1
0
[0 1 2 3 4]
0
(96000, 30, 9, 11)


In [10]:
# get rirs
mimo_srir = rirs[:, mic_idxs, src_idx, room_idx]
print(mimo_srir.shape)

# get annotation

curr_df = df.loc[
        (df['room_code'] == int(rooms[room_idx]))
    &   (df['src_id'] == src_idx+1)
    &   (df['array_id'] == arr_idx+1)
    &   (df['src_signal'] == 'chirp')
]
assert len(curr_df) == 5

# source pos
src_x = curr_df['src_pos_x'].unique()
src_y = curr_df['src_pos_y'].unique()
src_z = curr_df['src_pos_z'].unique()
assert len(src_x) == len(src_y) == len(src_z) == 1
src_pos = [[src_x, src_y, src_z]]

# array pos
arr_x = curr_df['array_bar_x'].unique()
arr_y = curr_df['array_bar_x'].unique()
arr_z = curr_df['array_bar_x'].unique()
assert len(arr_x) == len(arr_y) == len(arr_z) == 1
arr_pos = [[arr_x, arr_y, arr_z]]

# mics pos
mics_x = np.array(curr_df['mic_pos_x'])[None,:]
mics_y = np.array(curr_df['mic_pos_x'])[None,:]
mics_z = np.array(curr_df['mic_pos_x'])[None,:]
mics_pos = np.concatenate([mics_x, mics_y, mics_z], axis=0)

# echo note
curr_echo_df = echo_df.loc[(echo_df['arr'] == arr_idx) & (echo_df['src'] == src_idx)]
assert len(curr_echo_df) == 5
echo_toa_list = [
    np.array(curr_echo_df['direct'] )[None,:],
    np.array(curr_echo_df['floor']  )[None,:],
    np.array(curr_echo_df['ceiling'])[None,:],
    np.array(curr_echo_df['west']   )[None,:],
    np.array(curr_echo_df['south']  )[None,:],
    np.array(curr_echo_df['east']   )[None,:],
    np.array(curr_echo_df['north']  )[None,:]]
toas = np.concatenate(echo_toa_list, axis=0)

# rec filename
filename = curr_df['filename'].unique()[0]

# print(curr_df['room_rfl_floor'].unique()[0] > 0)
# print(curr_df['room_rfl_floor'].unique()[0] > 0)
# print(curr_df['room_rfl_ceiling'].unique()[0] > 0)
# print(curr_df['room_rfl_west'].unique()[0]  > 0)
# print(curr_df['room_rfl_south'].unique()[0] > 0)
# print(curr_df['room_rfl_east'].unique()[0]  > 0)
# print(curr_df['room_rfl_north'].unique()[0] > 0)
print(rooms[room_idx])

room_rfl = {
'f' : 'True' if curr_df['room_rfl_floor'].unique()[0] > 0 else 'False',
'c' : 'True' if curr_df['room_rfl_ceiling'].unique()[0] > 0 else 'False',
'w' : 'True' if curr_df['room_rfl_west'].unique()[0]  > 0 else 'False',
's' : 'True' if curr_df['room_rfl_south'].unique()[0] else 'False',
'e' : 'True' if curr_df['room_rfl_east'].unique()[0]  else 'False',
'n' : 'True' if curr_df['room_rfl_north'].unique()[0] else 'False',
'frn' : 'True' if curr_df['room_fornitures'].unique()[0] else 'False',
}

print(room_rfl)

(96000, 5)
000001
{'f': 'False', 'c': 'False', 'w': 'False', 's': 'False', 'e': 'False', 'n': 'True', 'frn': 'False'}


In [11]:
curr_df.columns

Index(['Unnamed: 0', 'filename', 'src_id', 'src_ch', 'src_type', 'src_signal',
       'src_pos_x', 'src_pos_y', 'src_pos_z', 'room_code', 'room_rfl_floor',
       'room_rfl_ceiling', 'room_rfl_west', 'room_rfl_south', 'room_rfl_east',
       'room_rfl_north', 'room_fornitures', 'room_temperature',
       'rec_silence_dB', 'rec_artifacts', 'mic_type', 'mic_id', 'mic_ch',
       'mic_pos_x', 'mic_pos_y', 'mic_pos_z', 'array_id', 'array_bar_x',
       'array_bar_y', 'array_bar_z'],
      dtype='object')

In [15]:
sofa = sf.Sofa('SingleRoomSRIR')

print(sofa.GLOBAL_Conventions)
print(sofa.GLOBAL_SOFAConventions)
print(sofa.GLOBAL_SOFAConventionsVersion)
print(sofa.GLOBAL_DataType)

sofa.GLOBAL_DatabaseName = 'dEchorate'
print(sofa.GLOBAL_DatabaseName)

sofa.GLOBAL_RoomType = 'shoebox'
print(sofa.GLOBAL_RoomType)

title_suffix = f'room:{rooms[room_idx]}_src:{src_idx+1}_arr:{arr_idx+1}_mics:{mic_idxs.min()+1}-{mic_idxs.max()+1}'
sofa.GLOBAL_Title = f'dEchorate_{title_suffix}'
print(sofa.GLOBAL_Title)

sofa.GLOBAL_DateCreated = '2022-04-20'
print(sofa.GLOBAL_DateCreated)
sofa.GLOBAL_DateModified = sofa.GLOBAL_DateCreated
print(sofa.GLOBAL_DateModified)

print(sofa.GLOBAL_APIName)
print(sofa.GLOBAL_APIVersion)

sofa.GLOBAL_AuthorContact = 'diego.dicarlo89@gmail.com'
print(sofa.GLOBAL_AuthorContact)

sofa.GLOBAL_Organization = 'INRIA Rennes (Fr)'
print(sofa.GLOBAL_Organization)

sofa.GLOBAL_License = 'The MIT License (MIT) Copyright (c) 2019, Diego Di Carlo. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.'
print(sofa.GLOBAL_License)

sofa.GLOBAL_ApplicationName = 'Python'
print(sofa.GLOBAL_ApplicationName)

import platform
sofa.GLOBAL_ApplicationVersion = platform.python_version()
print(sofa.GLOBAL_ApplicationVersion)


sofa.GLOBAL_Comment = """- website: 'http://diegodicarlo.com/datasets/dechorate';
- data: 'https://zenodo.org/record/5562386';
- code: 'https://github.com/Chutlhu/DechorateDB'; 
- paper: 'https://rdcu.be/cLJIG'"""
print(sofa.GLOBAL_Comment)

sofa.GLOBAL_RoomDescription = 'Recorded at the Bar\'Ilan acoustic lab. Refer to the publication for all the details.'
print(sofa.GLOBAL_RoomDescription)

sofa.GLOBAL_History = """
- 2019-01-20: Recorded raw data
- 2019-06-17: Processed annotation
- 2021-03-22: Publication on Zenodo v0
- 2022-04-21: Creation SOFA"""

print(sofa.GLOBAL_History)

sofa.GLOBAL_References = """@article{dicarlo2021dechorate,
  title={dEchorate: a calibrated room impulse response dataset for echo-aware signal processing},
  author={Carlo, Diego Di and Tandeitnik, Pinchas and Foy, Cedri{\'c} and Bertin, Nancy and Deleforge, Antoine and Gannot, Sharon},
  journal={EURASIP Journal on Audio, Speech, and Music Processing},
  volume={2021},
  number={1},
  pages={1--15},
  year={2021},
  publisher={Springer}
}"""
print(sofa.GLOBAL_References)

# sofa.GLOBAL_Origin = ''
# print(sofa.GLOBAL_Origin)

## --- RIRS --- ##
sofa.Data_IR = mimo_srir.T[None,:,:,None] # must be in M x R x N x E (measurements, receivers, samples, emitters)
assert sofa.Data_IR.shape == (1,5,2*Fs,1)
# print(sofa.Data_IR)
sofa.Data_SamplingRate = Fs
# print(sofa.Data_SamplingRate)
sofa.Data_SamplingRate_Units = 'hertz'
# print(sofa.Data_SamplingRate_Units)
sofa.Data_Delay = [0] * 5
# print(sofa.Data_Delay)

## --- RECORDINGS --- ##
sofa.add_attribute("GLOBAL_RecordingFilenames", filename)

## --- ARRAY --- ##

# Listener = compact microphone array (it may contain many receiver, i.e. mics)
sofa.GLOBAL_ListenerShortName = f'array_id:{arr_idx}'
# print(sofa.GLOBAL_ListenerShortName)

sofa.GLOBAL_ListenerDescription = 'handcrafted 5-mics non-Uniform Linear Array'
# print(sofa.GLOBAL_ListenerDescription)

sofa.ListenerPosition = arr_pos
# print(sofa.ListenerPosition)
sofa.ListenerPosition_Type = 'cartesian'
# print(sofa.ListenerPosition_Type)
sofa.ListenerPosition_Units = 'metre'
# print(sofa.ListenerPosition_Units)

sofa.ListenerView = [[1, 0, 0]]
# print(sofa.ListenerView)
sofa.ListenerUp =  [[1, 0, 0]]
# print(sofa.ListenerUp)
sofa.ListenerView_Type = 'cartesian'
# print(sofa.ListenerView_Type)
sofa.ListenerView_Units = 'metre'
# print(sofa.ListenerView_Units)

## --- MICROPHONES --- ##

# Receiver = MICROPHONE
sofa.GLOBAL_ReceiverShortName = 'capsule'
# print(sofa.GLOBAL_ReceiverShortName)

sofa.GLOBAL_ReceiverDescription = 'AKG CK32'
# print(sofa.GLOBAL_ReceiverDescription)

# print(mics_pos.shape)
sofa.ReceiverPosition = mics_pos.T
# print(sofa.ReceiverPosition)
sofa.ReceiverPosition_Type = 'cartesian'
# print(sofa.ReceiverPosition_Type)
sofa.ReceiverPosition_Units = 'metre'
# print(sofa.ReceiverPosition_Units)
sofa.ReceiverView = [[1,0,0]]*5
# print(sofa.ReceiverView)
sofa.ReceiverUp = [[1,0,0]]*5
# print(sofa.ReceiverUp)
sofa.ReceiverView_Type = 'cartesian'
# print(sofa.ReceiverView_Type)
sofa.ReceiverView_Units = 'metre'
# print(sofa.ReceiverView_Units)


# --- SOURCES --- ## 

# Source = compact sound source (it may contain more emitters, i.e. speakers)
sofa.GLOBAL_SourceShortName = f'loudspeaker_id:{src_idx}'
print(sofa.GLOBAL_SourceShortName)

sources_description = ['Directional Avanton MIXCUBE']*6 + ['Omnidirectional B&G']*3

sofa.GLOBAL_SourceDescription = sources_description[src_idx]
print(sofa.GLOBAL_SourceDescription)

sofa.SourcePosition = src_pos
print(sofa.SourcePosition)
sofa.SourcePosition_Type = 'cartesian'
print(sofa.SourcePosition_Type)
sofa.SourcePosition_Units = 'metre'
print(sofa.SourcePosition_Units)
sofa.SourceView = [1,0,0]
print(sofa.SourceView)
sofa.SourceUp = [1,0,0]
print(sofa.SourceUp)
sofa.SourceView_Type = 'cartesian'
print(sofa.SourceView_Type)
sofa.SourceView_Units = 'metre'
print(sofa.SourceView_Units)

# # for us, emitter = source 

# sofa.GLOBAL_EmitterShortName = sofa.GLOBAL_SourceShortName
# # print(sofa.GLOBAL_EmitterShortName)
# sofa.GLOBAL_EmitterDescription = sofa.GLOBAL_SourceDescription
# # print(sofa.GLOBAL_EmitterDescription)

# sofa.EmitterPosition = sofa.SourcePosition
# # print(sofa.EmitterPosition)
# sofa.EmitterPosition_Type = sofa.SourcePosition_Type
# # print(sofa.EmitterPosition_Type)
# sofa.EmitterPosition_Units = sofa.SourcePosition_Units
# # print(sofa.EmitterPosition_Units)
# sofa.EmitterView = sofa.SourceView
# # print(sofa.EmitterView)
# sofa.EmitterUp = sofa.SourceUp
# # print(sofa.EmitterUp)
# sofa.EmitterView_Type = sofa.EmitterView_Type
# # print(sofa.EmitterView_Type)
# sofa.EmitterView_Units = sofa.EmitterView_Units
# # print(sofa.EmitterView_Units)

## --- ROOM --- ##

sofa.RoomCornerA = [0,0,0]
# print(sofa.RoomCornerA)
sofa.RoomCornerB = constants['room_size']
# print(sofa.RoomCornerB)
sofa.RoomCorners_Type = 'cartesian'
# print(sofa.RoomCorners_Type)
sofa.RoomCorners_Units = 'metre'
# print(sofa.RoomCorners_Units)


sofa.add_variable("ReflectiveSurfacesCode", rooms[room_idx], "string", "S")
sofa.add_attribute("ReflectiveSurfacesCode_Order", "Floor,West,South,East,North")

sofa.add_variable("RoomFloorIsRefective",    [room_rfl['f']], "string", "S")
sofa.add_variable("RoomCleilingIsRefective", [room_rfl['c']], "string", "S")
sofa.add_variable("RoomWestIsRefective",     [room_rfl['w']], "string", "S")
sofa.add_variable("RoomSouthIsRefective",    [room_rfl['s']], "string", "S")
sofa.add_variable("RoomEastIsRefective",     [room_rfl['e']], "string", "S")
sofa.add_variable("RoomNorthIsRefective",    [room_rfl['n']], "string", "S")
sofa.add_variable("RoomHasFornitures",       [room_rfl['frn']], "string", "S")

sofa.add_variable("Temperature", constants['room_temperature'], "double", "I")
sofa.add_attribute("Temperature_Units", "degree Celsius")
sofa.add_variable("RelativeHumidity", constants['room_humidity'], "double", "I")
sofa.add_attribute("RelativeHumidity_Units", "percentage")
sofa.add_variable("SpeedOfSound", constants['speed_of_sound'], "double", "I")
sofa.add_attribute("SpeedOfSound_Units", "metre/seconds")
print(sofa.SpeedOfSound)

## --- ECHOES --- ##
sofa.add_variable("EchoTimingsPeakPicking", toas, "double", "KR")
sofa.add_attribute("EchoTimingsPeakPicking_Units", "samples")

SOFA
SingleRoomSRIR
1.0
FIR
dEchorate
shoebox
dEchorate_room:000001_src:1_arr:1_mics:1-5
2022-04-20
2022-04-20
sofar SOFA API for Python (pyfar.org)
0.3.1
diego.dicarlo89@gmail.com
INRIA Rennes (Fr)
The MIT License (MIT) Copyright (c) 2019, Diego Di Carlo. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AN

/home/chutlhu/Documents/PhD/Code/DechorateDB/venv/lib/python3.8/site-packages/sofar/sofar.py:571: UserWarning: Added custom dimension KR to SOFA object
  warnings.warn(


In [16]:
sofa.verify()
sofa.list_dimensions

E = 1 emitter (set by EmitterPosition of dimension ECI, ECM)
M = 1 measurements (set by Data_IR of dimension MRN)
R = 5 receiver (set by Data_IR of dimension MRN)
N = 96000 samples (set by Data_IR of dimension MRN)
K = 7 custom dimension (set by EchoTimingsPeakPicking of dimension KR)
C = 3 coordinate dimensions, fixed
I = 1 single dimension, fixed
S = 6 maximum string length



In [19]:
filename = sofa.GLOBAL_Title
path_to_out = f'../outputs/dEchorate_sofa/{filename}.sofa'
path_to_out = 'prova.sofa'
print(path_to_out)
sf.write_sofa(path_to_out, sofa, compression=7)

prova.sofa
<class 'netCDF4._netCDF4.Variable'>
float64 ListenerPosition(M, C)
unlimited dimensions: 
current shape = (1, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 ListenerView(I, C)
unlimited dimensions: 
current shape = (1, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 ListenerUp(I, C)
unlimited dimensions: 
current shape = (1, 3)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 ReceiverPosition(R, C, I)
unlimited dimensions: 
current shape = (5, 3, 1)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 ReceiverView(R, C, I)
unlimited dimensions: 
current shape = (5, 3, 1)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 ReceiverUp(R, C, I)
unlimited dimensions: 
current shape = (5, 3, 1)
filling 

ValueError: shape mismatch: objects cannot be broadcast to a single shape